# Lecture 16: Writing Functions in R
<div style="border: 1px double black; padding: 10px; margin: 10px">

**After today's lecture you will understand:**
* How to write functions in R
</div>

These notes correspond to Chapter 26 of the book.





In [1]:
library(tidyverse)
remotes::install_github("bradleyboehmke/harrypotter")
install.packages("tidytext")
library(harrypotter)
library(tidytext)

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.4.1     ✔ purrr   1.0.1
✔ tibble  3.2.0     ✔ dplyr   1.1.0
✔ tidyr   1.3.0     ✔ stringr 1.4.1
✔ readr   2.1.4     ✔ forcats 1.0.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()




── R CMD build ─────────────────────────────────────────────────────────────────
* checking for file ‘/tmp/RtmpXRCotN/remotes46c5249d3e8/bradleyboehmke-harrypotter-51f7146/DESCRIPTION’ ... OK
* preparing ‘harrypotter’:
* checking DESCRIPTION meta-information ... OK
* checking for LF line-endings in source and make files and shell scripts
* checking for empty or unneeded directories
* building ‘harrypotter_0.1.0.tar.gz’



Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘Rcpp’, ‘SnowballC’, ‘janeaustenr’, ‘tokenizers’




# Function
This lecture will be about writing our own functions. 
- Functions are not new to us; we have been using them since the very first lecture.
- `print`, `filter`, `mutate`, `summarize`, `tibble`, etc.: all are functions.

In [1]:
class(filter)

[1] "function"

### Why write out own functions??

Often when programming we find ourselves repeating the same block of code with minor modifications. 

Did you encounter this situation in your HW7 when you applied the same logic again and again, while deriving the sentiment score for all the Harry Potter books??

That is precisely when a function is defined to save ourselves from repetition!

Let us start with simple examples. When building machine learning models (which you will learn next week) it is common practice to normalize all the columns values to the same scale; typically between 0 and 1. Let us take the `mpg` dataset and see its current min and max values:

In [17]:
mpg$hwy %>% range

[1] 12 44

Let us now normalize it:

In [18]:
hwy_a <- (mpg$hwy - min(mpg$hwy, na.rm = TRUE)) / (max(mpg$hwy, na.rm = TRUE) - min(mpg$hwy, na.rm = TRUE))
hwy_a %>% range

[1] 0 1

We need to do the exact same for all columns; to begin with let us do for all numerical columns. And it is easy to copy and paste the same code from above and change the variable names

In [25]:
cty_a <- (mpg$cty - min(mpg$cty, na.rm = TRUE)) / (max(mpg$cty, na.rm = TRUE) - min(mpg$hwy, na.rm = TRUE))
cty_a %>% range

[1] 0.000000 1.130435

## 🤔 Quiz

Why is the range not showing a maximum of 1?

&nbsp;A. For the `cty` values it cannot have a maximum value of 1<br/>
&nbsp;B. The numerator should use `max` function<br/>
&nbsp;C. Copy-paste led to making an accidental mistake in the formula.<br/>

We still need to do the exact same procedure for all other columns as well and so intead of repeating ourselves we can introduce a function to do the repeats.

[1] 0 1
[1] 0 1
[1] 0 1


## Anatomy of a function
To write a function we should first think about the inputs and output. A function takes input(s), does something(s) to them, and then returns an output.


    f <- function(input) {
        <do some things>
        return(output)
    }
    output <- f(input)



In [ ]:
rescale <- function(x){
    (x - min(x, na.rm = T)) / (max(x, na.rm = T) - min(x, na.rm = T))
}

In R (and unlike Python, C++, Java, or other languages), it is not necessary to use a `return` keyword to specify the return value of a function. The return value of a function is simply the value of the last expression evaluated within the function body. 

## 🤔 Quiz

What is the output of our `rescale` function?

* A. x
* B. nothing
* C. rescaled vector values



### Arguments

Functions will often have multiple arguments. Some arguments have default values, others do not. All arguments without default values must be passed to a function. Arguments can be passed by name or position. For instance, 




In [35]:
# generate 5 numbers from a Normal(0, 1) distribution.
w = rnorm(5, mean = 0, sd = 1)
x = rnorm(n = 5, mean = 0, sd = 1)
y = rnorm(5, 0, 1)
z = rnorm(5)
round(cbind(w, x, y, z), 1)

w,x,y,z
0.0,1.5,-1.3,-0.6
-2.8,0.0,0.1,-0.9
-0.2,1.3,-0.2,-1.2
-0.8,0.7,-0.4,-0.7
-0.5,-1.2,-0.1,0.7


Arguments passed by name need not be in order:

In [34]:
w = rnorm(mean = 0, sd = 1, n = 5)
u = rnorm(mean = 0, sd = 1, 5) # This also works but is bad style. 
round(rbind(u = u, w = w), 1)

# unnamed arguments get passed to the first argument after the names arguments are assigned

u,0.2,-0.1,1.8,-1.2,-0.1
w,1.7,-1.2,0.5,-1.2,-0.1


# z-score function

Let us create a function to compute z-scores of a vector

In [39]:
# function to compute z-scores
z_score1 = function(x) {
  #inputs: x - a numeric vector
  #outputs: the z-scores for x
  xbar = mean(x)
  s = sd(x)
  z = (x - mean(x)) / s
  return(z)  
}

z_score1

function(x) {
  #inputs: x - a numeric vector
  #outputs: the z-scores for x
  xbar = mean(x)
  s = sd(x)
  z = (x - mean(x)) / s
  return(z)  
}

The return statement is not strictly necessary, but can make complex functions more readable. It is good practice to avoid creating intermediate objects to store values only used once.



In [40]:
# function to compute z-scores
z_score2 = function(x){
  #inputs: x - a numeric vector
  #outputs: the z-scores for x
  (x - mean(x)) / sd(x)
}

In [ ]:
x = rnorm(10, 3, 1) ## generate some normally distributed values
round( cbind(x, 'Z1' = z_score1(x), 'Z2' = z_score2(x) ), 1)

x,Z1,Z2
3.0,-0.3,-0.3
3.2,0.0,0.0
2.9,-0.5,-0.5
3.4,0.3,0.3
3.2,-0.1,-0.1
2.2,-1.5,-1.5
2.8,-0.6,-0.6
4.9,2.4,2.4
3.3,0.1,0.1
3.3,0.1,0.1


### Default Parameters

We can set default values for parameters using the construction `parameter = xx` in the function definition.




In [ ]:
# function to compute z-scores
z_score3 = function(x, na.rm = T){
  {x - mean(x, na.rm = na.rm)} / sd(x, na.rm = na.rm)
}

In [ ]:
x = c(NA, x, NA)
round( cbind(x, 'Z1' = z_score1(x), 'Z2' = z_score2(x), 'Z3' = z_score3(x) ), 1)

x,Z1,Z2,Z3
NA,NA,NA,NA
3.0,NA,NA,-0.3
3.2,NA,NA,0.0
2.9,NA,NA,-0.5
3.4,NA,NA,0.3
3.2,NA,NA,-0.1
2.2,NA,NA,-1.5
2.8,NA,NA,-0.6
4.9,NA,NA,2.4
3.3,NA,NA,0.1


## Scope

Scoping refers to how R looks up the value associated with an object referred to by name. There are two types of scoping – lexical and dynamic – but we will concern ourselves only with lexical scoping here. There are four keys to understanding scoping:

- environments
- name masking
- variables vs functions
- dynamic look up 


An environment can be thought of as a context in which names are associated with objects. Each time a function is called, it generates a new environment for the computation.

Consider the following examples:

In [42]:
ls()

[1] "a"          "b"          "c"          "cty"        "cty_a"     
 [6] "hwy_a"      "rescale"    "sum"        "u"          "w"         
[11] "x"          "x_centered" "y"          "z"          "z_score1"  
[16] "z_score2"

In [43]:
f1 = function() {
  f1_message = "I'm defined inside of f!"  # `message` is a function in base
  ls()
}
f1()

[1] "f1_message"

In [44]:
exists('f1') # f1 %in% ls() 

[1] TRUE

In [45]:
exists('f1_message')

[1] FALSE

In [46]:
environment() # here we are in the global environment

<environment: R_GlobalEnv>

In [47]:
f2 = function(){
  environment() # here we are in the local environment -- each time we get a different local environment
    # created for the purpose of this function
}
f2()

<environment: 0x564d05c250a0>

In [ ]:
rm(f1, f2)

Name masking refers to where and in what order `R` looks for object names.
When we call `f1` above, `R` first looks in the current environment which happens to be the global environment. The call to `ls()` however, happens within the environment created by the function call and hence returns only the objects defined in the local environment.

When an environment is created, it gets nested within the current environment referred to as the “parent environment”. When an object is referenced we first look in the current environment and move recursively up through parent environments until we find a value bound to that name.



Name masking refers to the notion that objects of the same name can exist in different environments. Consider these examples:



In [57]:
#  Example 3 -- lexical scoping
y = x = 'I came from outside of f!'
f3 = function(){
  x =  'I came from inside of f!'
  print(paste("x =", x, "and y =", y))
}
f3()
print(paste("outside-x =", x, "and outside-y =", y))

[1] "x = I came from inside of f! and y = I came from outside of f!"
[1] "outside-x = I came from outside of f! and outside-y = I came from outside of f!"


* x is redefined inside the function enviornment
* y is not, so R will search for y in the parent environment and keep moving up
* x that is associated with f3, is not going to change the x in the global environment, unless we explicitly write the code to do that

In [ ]:
#  Example 4 -- masking
mean = function(x){ 
    sum(x)
}
mean(1:10)

[1] 55

In [ ]:
base::mean(1:10)



[1] 5.5

In [ ]:
rm(mean)

R also uses dynamic look up, meaning values are searched for when a function is called, not when it is created. In the example above, y was defined in the global environment rather than within the function body. This means the value returned by f3 depends on the value of y in the global environment. You should generally avoid this, but there are occasions where it can be useful.



In [ ]:
# Example 5 - dynamic lookup
y = "I have been reinvented!"
f3()

$x
[1] "I came from inside of f!"

$y
[1] "I have been reinvented!"

## Dataframe functions

A case of a problem of `indirection`, as `dplyr` uses `tidy evaluation` to allow you to refer to the names of variables inside your data frame without any special treatment.

In [70]:
grouped_mean <- function(df, group_var, mean_var) {
  df |> 
    group_by(group_var) |> 
    summarize(mean(mean_var))
}

grouped_mean(mpg, model, hwy)

ERROR: ignored

### Fix with embracing `{{ }}`

In [76]:
grouped_mean <- function(df, group_var, mean_var) {
  df |> 
    group_by({{ group_var }}) |> 
    summarize(mean({{ mean_var }}))
}

grouped_mean(mpg, model, hwy) %>% head

model,mean(hwy)
<chr>,<dbl>
4runner 4wd,18.83333
a4,28.28571
a4 quattro,25.75000
a6 quattro,24.00000
altima,28.66667
c1500 suburban 2wd,17.80000


In [101]:
grouped_mean <- function(df, year, group_var, mean_var) {
  df %>% filter(year == {{ year }}) %>%
    group_by({{ group_var }}) |> 
    summarize(mean({{ mean_var }}))
}

grouped_mean(mpg, 1999, model, hwy) %>% head

model,mean(hwy)
<chr>,<dbl>
4runner 4wd,19.00
a4,27.50
a4 quattro,25.25
a6 quattro,24.00
altima,28.00
c1500 suburban 2wd,17.00


### When to embrace?🤔

There are two terms to look for in the docs which correspond to the two most common sub-types of tidy evaluation:

* Data-masking: this is used in functions like arrange(), filter(), and summarize() that compute with variables.

* Tidy-selection: this is used for functions like select(), relocate(), and rename() that select variables.

In [77]:
?select

In [79]:
?summarize

## Pick function

Applied for verbs that use data-masking when sub-selecting the columns

In [114]:
?pick

In [97]:
mean_hwy <- function(df, var1, var2, var_3) {
  df |> 
    group_by({{ var1 }}, {{ var2 }}) |> 
    summarize(
      mean = mean(is.na({{ var_3 }})),
      .groups = "drop"
    )
}

mpg |> 
  mean_hwy(manufacturer, model, hwy) %>% head

manufacturer,model,mean
<chr>,<chr>,<dbl>
audi,a4,0
audi,a4 quattro,0
audi,a6 quattro,0
chevrolet,c1500 suburban 2wd,0
chevrolet,corvette,0
chevrolet,k1500 tahoe 4wd,0


In [112]:
mean_hwy <- function(df, var1, var2) {
  df |> 
    group_by(pick({{ var1 }})) |> 
    summarize(
      mean = mean(is.na({{ var2 }})),
      .groups = "drop"
    )
}

mpg |> 
  mean_hwy(c(manufacturer, model), hwy) %>% head

manufacturer,model,mean
<chr>,<chr>,<dbl>
audi,a4,0
audi,a4 quattro,0
audi,a6 quattro,0
chevrolet,c1500 suburban 2wd,0
chevrolet,corvette,0
chevrolet,k1500 tahoe 4wd,0
